In [33]:
import os
import pickle as pkl
import pandas as pd
import seaborn as sns
sns.set_theme('talk', palette=sns.color_palette('Set2'))
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import cartopy
import plotly.graph_objects as go
import numpy as np
from sqlalchemy import create_engine
pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 150

In [34]:
data = pd.read_csv('../1_web_scraping/apartment_details.csv')

In [35]:
data

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url
0,Zabrze,Centrum,śląskie,50.311850,18.769910,wtorny,sprzedaz,59.76,3.0,parter,1.0,1950.0,199000,https://gratka.pl/nieruchomosci/mieszkanie-zabrze-centrum-joanny-znienskiej/ob/36860645
1,Łódź,Górna,łódzkie,51.728420,19.496120,wtorny,sprzedaz,57.47,3.0,4,4.0,1964.0,428000,https://gratka.pl/nieruchomosci/mieszkanie-lodz-gorna-slaska/ob/34538901
2,Warszawa,Praga-Północ,mazowieckie,52.273618,21.011744,pierwotny,sprzedaz,79.43,4.0,2,8.0,2025.0,Zapytajoce,https://gratka.pl/nieruchomosci/mieszkanie-warszawa-praga-polnoc-a-kotsisa-2/ob/31680021
3,Bytom,NaN,śląskie,50.343530,18.917150,wtorny,sprzedaz,92.00,3.0,3,3.0,1920.0,275000,https://gratka.pl/nieruchomosci/mieszkanie-bytom-jagiellonska/ob/36905723
4,Barwice,NaN,zachodniopomorskie,53.738694,16.354473,wtorny,sprzedaz,60.60,3.0,parter,1.0,NaN,120000,https://gratka.pl/nieruchomosci/mieszkanie-szczecinecki-barwice-boleslawa-chrobrego/ob/33933797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Oława,NaN,dolnośląskie,50.944309,17.294559,wtorny,sprzedaz,78.36,3.0,2,2.0,1940.0,495000,https://gratka.pl/nieruchomosci/mieszkanie-olawski-olawa-pl-zamkowy/ob/34322621
4996,Piekary Śląskie,NaN,śląskie,50.375500,18.944799,wtorny,sprzedaz,47.90,2.0,1,3.0,1988.0,279999,https://gratka.pl/nieruchomosci/mieszkanie-piekary-slaskie-janty/ob/35011749
4997,Gdynia,Śródmieście,pomorskie,54.523480,18.542800,wtorny,sprzedaz,159.90,5.0,6,6.0,2023.0,4259600,https://gratka.pl/nieruchomosci/mieszkanie-gdynia-srodmiescie-jana-z-kolna/ob/35334157
4998,Wrocław,Śródmieście,dolnośląskie,51.115600,17.058260,wtorny,sprzedaz,110.00,4.0,1,4.0,NaN,1270000,https://gratka.pl/nieruchomosci/mieszkanie-wroclaw-srodmiescie-mikolaja-reja/ob/34643097


In [37]:
data.dropna(subset='city', inplace=True)

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4912 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            4912 non-null   object 
 1   district        3376 non-null   object 
 2   voivodeship     4912 non-null   object 
 3   localization_y  4908 non-null   float64
 4   localization_x  4908 non-null   float64
 5   market          4912 non-null   object 
 6   offer_type      4912 non-null   object 
 7   area            4912 non-null   float64
 8   rooms           4901 non-null   float64
 9   floor           4867 non-null   object 
 10  floors          4259 non-null   float64
 11  build_yr        4242 non-null   float64
 12  price           4912 non-null   object 
 13  url             4912 non-null   object 
dtypes: float64(6), object(8)
memory usage: 575.6+ KB


In [44]:
data[data.city.str.startswith('a-z')]

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url


In [45]:
def starts_small(x):
    if x[0].islower():
        return True
    return False

In [47]:
data[data.city.apply(starts_small)]

,city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url
469,karkonoski,Szklarska Poręba,dolnośląskie,50.838155,15.548648,wtorny,sprzedaz,45.30,2.0,1,2.0,NaN,679000,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-szklarska-poreba-ludwika-warynskiego/ob/36396837
879,karkonoski,Szklarska Poręba,dolnośląskie,50.824041,15.522704,wtorny,sprzedaz,30.00,1.0,6,6.0,2019.0,489000,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-szklarska-poreba/ob/35025541
923,karkonoski,Karpacz,dolnośląskie,50.773737,15.738665,pierwotny,sprzedaz,63.75,3.0,2,3.0,2025.0,784125,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-karpacz-karkonoska/ob/35413875
1216,karkonoski,Szklarska Poręba,dolnośląskie,50.824330,15.526234,wtorny,sprzedaz,32.00,1.0,1,7.0,2018.0,465000,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-szklarska-poreba-gorna/ob/34875089
1267,karkonoski,Szklarska Poręba,dolnośląskie,50.824041,15.522704,pierwotny,sprzedaz,34.12,2.0,1,4.0,2024.0,740109,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-szklarska-poreba/ob/34174517
1412,karkonoski,Szklarska Poręba,dolnośląskie,50.831670,15.520590,wtorny,sprzedaz,32.95,1.0,6,6.0,2019.0,684000,https://gratka.pl/nieruchomosci/mieszkanie-karkonoski-szklarska-poreba-gorna/ob/36684335
4067,ul.,NaN,mazowieckie,51.990600,21.026700,wtorny,sprzedaz,128.80,4.0,3,3.0,2002.0,1660000,https://gratka.pl/nieruchomosci/mieszkanie-piaseczynski/ob/35344741
